# Master Thesis Script
## Twitter Network Data - Friends

Extract politicians' friends' follower counts, create dictionary with politician IDs, and their friends' IDs and follower counts.

In [1]:
import pandas as pd
import json as json
import numpy as np
import regex as re
import matplotlib.pyplot as plt
from math import log
import seaborn as sns
from statistics import mean
import random
from scipy.stats import powerlaw
import scipy
#from pyvis.network import Network
#import networkx as nx
import requests
from requests import get
import time
import csv
import gzip
import shutil
import os
from tqdm import tqdm

In [2]:
# make list of document names to iterate through
# folder path
dir_path = r'Json_Data_Friends'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path[:-6])

In [30]:
print(res)

['21572351', '581141508', '1345807954604412928', '23725826', '305216911', '1074129612', '20546536', '240760644', '254082173', '281540744', '1080695666760929024', '818472418620608512', '237763317', '1201953403099893760', '1080584229510172678', '76069325', '50055701', '113763245', '229197216', '733672650120253440', '24745957', '1135486501', '239949176', '1079770852302958976', '1067541214671577093', '914815520842616832', '28602948', '242926427', '1052896620797460481', '224285242', '256157258', '574799823', '14140370', '2847603597', '38285038', '40302336', '1339931490', '304138251', '1072158357237174016', '1080978331535896960', '816719802328715264', '16256269', '4049197462', '15764644', '15745368', '49217025', '1122934522134315008', '278094476', '1344389506963808264', '22044727', '1340783304304410625', '1021216905795899008', '162069635', '432895323', '63650107', '158470209', '94154021', '839226269627621376', '38960144', '915975047545749504', '33576489', '499268312', '894940941039144960', '

In [3]:
# Opening JSON file
f = open(r'Twitter_authors_Thesis_Updated.json')
  
# returns JSON object as 
# a dictionary
data_authors = pd.read_json(f)
  
# Closing file
f.close()

In [4]:
data_authors

,author_id,author.username,author.name,author_id_strng
0,211530910,RepBillJohnson,Rep. Bill Johnson,211530910
1,81207735,JohnsonLeads,Bill Johnson,81207735
2,219429281,RepKinzinger,Adam Kinzinger,219429281
3,231510077,JoaquinCastrotx,Joaquin Castro,231510077
4,233693291,RepRickCrawford,Rep. Rick Crawford,233693291
...,...,...,...,...
901,163570705,repcleaver,Rep. Emanuel Cleaver,163570705
902,168502762,RepEBJ,Rep. Eddie Bernice Johnson,168502762
903,188403766,Foster4Congress,Bill Foster,188403766
904,193732179,RepJudyChu,Judy Chu,193732179


In [5]:
authors_lst = data_authors["author_id_strng"].tolist()

In [6]:
res_int = [int(item) for item in res]

In [32]:
print(res_int)

[21572351, 581141508, 1345807954604412928, 23725826, 305216911, 1074129612, 20546536, 240760644, 254082173, 281540744, 1080695666760929024, 818472418620608512, 237763317, 1201953403099893760, 1080584229510172678, 76069325, 50055701, 113763245, 229197216, 733672650120253440, 24745957, 1135486501, 239949176, 1079770852302958976, 1067541214671577093, 914815520842616832, 28602948, 242926427, 1052896620797460481, 224285242, 256157258, 574799823, 14140370, 2847603597, 38285038, 40302336, 1339931490, 304138251, 1072158357237174016, 1080978331535896960, 816719802328715264, 16256269, 4049197462, 15764644, 15745368, 49217025, 1122934522134315008, 278094476, 1344389506963808264, 22044727, 1340783304304410625, 1021216905795899008, 162069635, 432895323, 63650107, 158470209, 94154021, 839226269627621376, 38960144, 915975047545749504, 33576489, 499268312, 894940941039144960, 1082311988926124036, 169198625, 156045092, 1276403757224546304, 950758571821629441, 977247448820305920, 817076257770835968, 134

In [7]:
res_filtered = [item for item in res_int if item in authors_lst]

In [8]:
res_rest = []
for item in res_int:
    if item not in res_filtered:
        res_rest.append(item)

In [9]:
print(len(res_filtered))

818


In [10]:
print(res_filtered[0])

21572351


In [11]:
authors_notfound = []
for author in authors_lst:
    if author not in res_filtered:
           authors_notfound.append(author)

In [12]:
count = 0
for author in authors_notfound:
    author_str = str(author)
    short_author = int(author_str[:-4])
    for item in res_rest: 
        item_str = str(item)
        short_item = int(item_str[:-4])
        if short_author == short_item:
            res_filtered.append(item)
            count+=1
    #print("next author")
            
print(count)

135


In [13]:
dict_lst = []
not_working_lst = []
for j in tqdm (range(len(res_filtered)), desc="Loading..."):
#for item in res_filtered:
   # if (i == 779):
    #    not_working_lst.append(res[i])
     #   i=i+1
        
    
    with open(f'Json_Data_Friends/{res_filtered[j]}.jsonl', 'r') as json_file:
        json_list = list(json_file)
    
    for json_str in json_list:
        result = json.loads(json_str)
    
    # create empty dict for this politician to fill with data
    keyList = ["politician_id", "ids", "followers", "following"]
    dict_user = dict(zip(keyList, [None]*len(keyList)))
          
            
    try:       
        id_lst = []
        followers_lst = []
        following_lst = []
        
        for i in range(len(result["data"])):
            user = result["data"][i] 
            id_user = user["id"]
            followers = user["public_metrics"]["followers_count"]
            following = user["public_metrics"]["following_count"]
            
            id_lst.append(id_user)      
            followers_lst.append(followers)
            following_lst.append(following)
               
    
        dict_user["politician_id"] = res_filtered[j]
        dict_user["ids"] = id_lst
        dict_user["followers"] = followers_lst
        dict_user["following"] = following_lst
        
        dict_lst.append(dict_user)
    
    except:
        not_working_lst.append(res_filtered[j])

Loading...: 100%|██████████| 953/953 [00:18<00:00, 52.31it/s] 


In [14]:
print(len(not_working_lst))

98


In [15]:
print(len(dict_lst))

855


In [91]:
with open(f'Json_Data_Friends/{not_working_lst[40]}.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)

In [92]:
print(result)

{'errors': [{'value': '867495243482058752', 'detail': 'Could not find user with id: [867495243482058752].', 'title': 'Not Found Error', 'resource_type': 'user', 'parameter': 'id', 'resource_id': '867495243482058752', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}], '__twarc': {'url': 'https://api.twitter.com/2/users/867495243482058752/following?tweet.fields=attachments%2Cauthor_id%2Ccontext_annotations%2Cconversation_id%2Ccreated_at%2Centities%2Cgeo%2Cid%2Cin_reply_to_user_id%2Clang%2Cpublic_metrics%2Ctext%2Cpossibly_sensitive%2Creferenced_tweets%2Creply_settings%2Csource%2Cwithheld%2Cedit_controls&user.fields=created_at%2Cdescription%2Centities%2Cid%2Clocation%2Cname%2Cpinned_tweet_id%2Cprofile_image_url%2Cprotected%2Cpublic_metrics%2Curl%2Cusername%2Cverified%2Cwithheld&max_results=1000', 'version': '2.12.0', 'retrieved_at': '2023-04-21T19:40:14+00:00'}}


Missing friends information for 98 politicians.

In [16]:
with open(r'ExtractedData/Friends_ExtracedData_20230602.json', "w") as outfile:
    json.dump(dict_lst, outfile)